# Fig 6: Onset-Offset Comparisons
### Comparing Within-Detector Sensitivity to Onset vs. Offset (Fixations & Saccades Separately)

In [1]:
import os
import copy
import warnings
from typing import Optional

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import scikit_posthocs as sp
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.channel_sdt as ch_sdt

pio.renderers.default = "browser"
THRESHOLD = 5   # samples

## Load Data

In [2]:
fix_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=1,    # EventLabelEnum.FIXATION.value
    stimulus_type=STIMULUS_TYPE,
    threshold=THRESHOLD,
    channel_type=None,
)

sac_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=2,    # EventLabelEnum.SACCADE.value
    stimulus_type=STIMULUS_TYPE,
    threshold=THRESHOLD,
    channel_type=None,
)

# Remove unused metrics
fix_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)
sac_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)

# concatenate
metrics = pd.concat([fix_metrics, sac_metrics], keys=['fixation', 'saccade'])
metrics.index.names = [peyes.constants.EVENT_STR] + metrics.index.names[1:]
metrics = metrics.droplevel('threshold')

metrics

trial_id                                      25                      \
gt                                            RA                       
pred                                          MN   engbert  remodnav   
event    channel_type metric                                           
fixation onset        recall            0.785714  0.964286  0.285714   
                      precision         0.814815  0.818182  0.666667   
                      f1                0.800000  0.885246  0.400000   
                      false_alarm_rate  0.011752  0.014103  0.009402   
                      d_prime           3.056776  3.997164  1.783457   
                      criterion         0.736749  0.195839  1.457677   
         offset       recall            0.928571  1.000000  0.142857   
                      precision         0.962963  0.848485  0.333333   
                      f1                0.945455  0.918033  0.200000   
                      false_alarm_rate  0.002350  0.011752  0.018803   
                      d_prime           4.292082  5.048805  1.011544   
                      criterion         0.680807 -0.324197  1.573342   
saccade  onset        recall            1.000000  1.000000  0.520000   
                      precision         0.961538  0.806452  1.000000   
                      f1                0.980392  0.892857  0.684211   
                      false_alarm_rate  0.002334  0.014004  0.000000   
                      d_prime           5.459188  4.989576  2.898532   
                      criterion        -0.119005 -0.353811  1.399333   
         offset       recall            0.920000  0.520000  0.120000   
                      precision         0.884615  0.419355  0.230769   
                      f1                0.901961  0.464286  0.157895   
                      false_alarm_rate  0.007002  0.042011  0.023340   
                      d_prime           3.862237  1.777960  0.814211   
                      criterion         0.526047  0.838826  1.582092   

trial_id                                                              \
gt                                                                     
pred                                        idvt        nh       idt   
event    channel_type metric                                           
fixation onset        recall            0.035714  0.678571  0.035714   
                      precision         0.055556  0.703704  0.055556   
                      f1                0.043478  0.690909  0.043478   
                      false_alarm_rate  0.039957  0.018803  0.039957   
                      d_prime          -0.051561  2.542822 -0.051561   
                      criterion         1.776963  0.807703  1.776963   
         offset       recall            0.571429  0.500000  0.571429   
                      precision         0.888889  0.518519  0.888889   
                      f1                0.695652  0.509091  0.695652   
                      false_alarm_rate  0.004701  0.030556  0.004701   
                      d_prime           2.777103  1.872691  2.777103   
                      criterion         1.208539  0.936345  1.208539   
saccade  onset        recall            0.640000  0.480000  0.640000   
                      precision         0.941176  0.461538  0.941176   
                      f1                0.761905  0.470588  0.761905   
                      false_alarm_rate  0.002334  0.032676  0.002334   
                      d_prime           3.187557  1.792695  3.187557   
                      criterion         1.235320  0.946501  1.235320   
         offset       recall            0.160000  0.600000  0.160000   
                      precision         0.235294  0.576923  0.235294   
                      f1                0.190476  0.588235  0.190476   
                      false_alarm_rate  0.030342  0.025674  0.030342   
                      d_prime           0.881339  2.201913  0.881339   
                      criterion        

## Functions
### Linear Mixed Effects Model
We have a measurement (e.g., $d'$) for a hierarchy of conditions: Dataset (entire population) $\rightarrow$ GT Annotator $\rightarrow$ PRED Detector $\rightarrow$ Event (fixation/saccade) $\rightarrow$ Channel (onset/offset) $\rightarrow$ single (trial) measurement.  
  
**(1) Between-Detector Comparison:**  
For each **annotator** _(RA, MN)_ separately, we want to test the effect of **channel** _(onset/offset)_ and **event** _(fixation/saccade)_ and their interaction, across all **detectors**, on the measurement. We can do this using a linear mixed effects model (LME), using the `statsmodels` package. To put this in formula form:  
$$d' \sim \text{event} + \text{channel} + \text{event} \times \text{channel} + (1|\text{detector})$$
where the `(1|detector)` term specifies that the detector is a random effect.  
  
  
**(2) Within-Detector Comparison:**
For each **annotator-detector** pair, we want to test the effect of **channel** _(onset/offset)_ on the measurement. To put this in formula form:  
$$d' \sim \text{channel} + (1|\text{event})$$
where the `(1|event)` term specifies that the event (fixation/saccade) is a random effect.

In [3]:
def linear_mixed_effect(
        dataset: pd.DataFrame, metric: str, gt_annotator: str, pred_detector: Optional[str] = None, include_annotators: bool = True
):
    # extract the subset of data and reshape it from "wide" to "long" format
    subset = _extract_data(dataset, metric, gt_annotator, pred_detector, include_annotators)
    long_subset = _reshape_data(subset, metric)
    
    # create the LME model and fit to the long-format data
    if pred_detector:
        # within-detector comparison, use onset/offset as the fixed effect and fixation/saccade as the grouping variable
        formula = f"{metric} ~ {peyes.constants.CHANNEL_TYPE_STR}"  # fixed effect: onset/offset
        groups = peyes.constants.EVENT_STR                          # grouping variable: fixation/saccade
    else:
        # between-detector comparison, use fixation/saccade and onset/offset and their interaction as the fixed effect and the detector as grouping variable
        formula = f"{metric} ~ {peyes.constants.EVENT_STR} + {peyes.constants.CHANNEL_TYPE_STR} + {peyes.constants.EVENT_STR} * {peyes.constants.CHANNEL_TYPE_STR}"     # fixed effect: fixation/saccade, onset/offset, and their interaction
        groups = u.PRED_STR                # grouping variable: detector
    
    # add random intercept per group (detectors/events) and random slopes w.r.t. each fixed effect (events, channels)
    re_formula = "1 + " + formula.split(" ~ ")[1]
    
    # create and fit the model
    model = smf.mixedlm(formula, long_subset, groups=groups, re_formula=re_formula)
    result = model.fit()
    return result

def _extract_data(
        dataset: pd.DataFrame, metric: str, gt_annotator: str, pred_detector: Optional[str] = None, include_annotators: bool = True
) -> pd.DataFrame:
    """
    Extracts data for the given metric, GT annotator, and PRED detector, if provided.
    If pred_detector is None, the function returns the data for all detectors, including the GT annotators, unless `include_annotators` is set to False.
    
    dataset structure:
    - index contains the following levels: "event" (fixation/saccade), "channel_type" (onset/offset), "metric" (e.g. d_prime, f1, etc.)
    - columns contain the following levels: "trial_id" (numeric: 25, 26...), "gt" (RA/MN), "pred" (algorithmic detector)
    
    Returns a DataFrame with the following structure:  
    - Rows: MultiIndex with levels [pred, event, channel_type] if pred_detector is None, otherwise [event, channel_type]
    - Columns: Single-level index with trial_id
    - Values are the metric values for each trial for the (pred, event, channel_type) combination.
    
    Raise an AssertionError if the metric/annotator/detector is not found in the dataset.
    """
    all_metrics = dataset.index.get_level_values(peyes.constants.METRIC_STR).unique()
    assert metric in all_metrics, f"Metric '{metric}' not in {all_metrics}"
    all_annotators = dataset.columns.get_level_values(u.GT_STR).unique()
    assert gt_annotator in all_annotators, f"Annotator '{gt_annotator}' not in {all_annotators}"
    result = dataset.xs(metric, level=peyes.constants.METRIC_STR, axis=0, drop_level=True).xs(gt_annotator, level=u.GT_STR, axis=1, drop_level=True)
    result = result.stack(level=u.PRED_STR, future_stack=True).reorder_levels([u.PRED_STR, peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR])
    result.sort_index(inplace=True)
    if not include_annotators:
        is_annotator = result.index.get_level_values(u.PRED_STR).isin(all_annotators)
        result = result.loc[~is_annotator]
    
    # return the full subset if pred_detector is None
    if not pred_detector:
        return result
    # return the subset for the given pred_detector
    all_detectors = result.index.get_level_values(u.PRED_STR).unique()
    if pred_detector in all_detectors:
        return result.xs(pred_detector, level=u.PRED_STR, axis=0, drop_level=True)
    if pred_detector == "mean":
        return result.groupby(level=[peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]).mean()
    raise AssertionError(f"Unidentified Detector '{pred_detector}'")


def _reshape_data(data: pd.DataFrame, metric: str) -> pd.DataFrame:
    """ Reshape a DataFrame from "wide" to "long" format, using the metric as the value variable. """
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=FutureWarning)
        index_names, column_name = data.index.names, data.columns.name
        long_format_data = data.reset_index().melt(
            id_vars=index_names,
            var_name=column_name,
            value_name=metric
        )
    long_format_data.dropna(axis=0, inplace=True)   # remove rows with missing values
    return long_format_data

### Post-Hoc Dunn's Test
#### Examining detection differences **between** Detectors or **within** a Detector 
We compare detection performance between different event types (fixations/saccades) and channel types (onset/offset), across all detectors or for a specific detector (based on whether the `pred_detector` argument is provided or not). We use Dunn's post-hoc test to calculate pairwise differences, with the Bonferroni correction for multiple comparisons. The reason we use Dunn's test is that it is a non-parametric test that does not assume normality, and it is suitable for comparing multiple groups.

In [4]:
def posthoc_dunn(
        dataset, metric: str, gt_annotator: str, pred_detector: Optional[str], include_annotators: bool = True
) -> pd.DataFrame:
    pvalues = _calculate_pvalues(dataset, metric, gt_annotator, pred_detector, include_annotators)
    table = _fill_post_hoc_table(pvalues)
    if pred_detector:
        index_names = [peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]
    else:
        index_names = [u.PRED_STR, peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]
    new_index = pd.MultiIndex.from_tuples(table.index.map(lambda idx: tuple(idx.split('_'))), names=index_names)
    table.index = table.columns = new_index
    return table


def _calculate_pvalues(
        data: pd.DataFrame, metric: str, gt_annotator: str, pred_detector: Optional[str], include_annotators: bool = True
) -> pd.DataFrame:
    subset = _extract_data(data, metric, gt_annotator, pred_detector, include_annotators)
    long_subset = _reshape_data(subset, metric)
    if pred_detector:
        # within-detector comparison
        new_colname = f"{peyes.constants.EVENT_STR}_{peyes.constants.CHANNEL_TYPE_STR}"
        long_subset[new_colname] = long_subset[peyes.constants.EVENT_STR] + '_' + long_subset[peyes.constants.CHANNEL_TYPE_STR]
    else:
        # between-detector comparison
        new_colname = f"{u.PRED_STR}_{peyes.constants.EVENT_STR}_{peyes.constants.CHANNEL_TYPE_STR}"
        long_subset[new_colname] = long_subset[u.PRED_STR] + '_' + long_subset[peyes.constants.EVENT_STR] + '_' + long_subset[peyes.constants.CHANNEL_TYPE_STR]
    pvalues = sp.posthoc_dunn(long_subset, val_col=metric, group_col=new_colname, p_adjust='bonferroni')
    return pvalues


def _fill_post_hoc_table(
        posthoc_pvals: pd.DataFrame, alpha: float = 0.05, marginal_alpha: Optional[float] = 0.075,
) -> pd.DataFrame:
    assert 0 < alpha < 1, f"parameter `alpha` must be in range (0, 1), {alpha: .3f} given."
    if marginal_alpha is not None:
        assert alpha < marginal_alpha < 1, f"parameter `marginal_alpha` must be in range ({alpha: .3f}, 1), {marginal_alpha: .3f} given."
    table = np.full_like(posthoc_pvals, "n.s.", dtype=np.dtypes.StringDType())
    if marginal_alpha is not None:
        table[posthoc_pvals <= marginal_alpha] = '†'
    table[posthoc_pvals <= alpha] = '*'
    table[posthoc_pvals <= alpha / 5] = '**'
    table[posthoc_pvals <= alpha / 50] = '***'
    table = pd.DataFrame(table, index=posthoc_pvals.index, columns=posthoc_pvals.columns)
    
    for i, idx in enumerate(table.index):
        for j, col in enumerate(table.columns):
            if i == j:
                table.loc[idx, col] = '--'
            if i > j:
                table.iloc[i, j] = posthoc_pvals.iloc[j, i]
    return table

### Plotting

In [5]:
def single_bar_plot(
        data: pd.DataFrame, metric: str, gt_annotator: str, pred_detector: Optional[str] = None, include_annotators: bool = True
) -> go.Figure:
    subset = _extract_data(data, metric, gt_annotator, pred_detector, include_annotators)
    if not pred_detector:
        subset = subset.groupby(level=[peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]).mean()     # average across detectors
    summary = pd.concat([subset.mean(axis=1).rename("mean"), subset.std(axis=1).rename("std")], axis=1).reset_index()
    category_orders = {peyes.constants.CHANNEL_TYPE_STR: ["onset", "offset"]}   # order of the bars
    fig = px.bar(
        summary,
        x=peyes.constants.EVENT_STR, y="mean", error_y="std",
        color=peyes.constants.CHANNEL_TYPE_STR, category_orders=category_orders,
        labels={peyes.constants.EVENT_STR: "Event Type", "mean": "Mean", "std": "Standard Deviation"},
        barmode='group',
    )
    fig.update_layout(
        width=800, height=450,
        yaxis_title=metric.replace('_', ' ').title(),
    )
    return fig


def multiple_bar_plots(data: pd.DataFrame, metric: str, gt_annotator: str, include_annotators: bool = True) -> go.Figure:
    subset = _extract_data(data, metric, gt_annotator, include_annotators=include_annotators)
    detectors = sorted(
        subset.index.get_level_values(u.PRED_STR).unique(),
        key=lambda dett: LABELER_PLOTTING_CONFIG[dett][0]
    )
    overall_rows = 2
    detector_rows = 2 if len(detectors) > 4 else 1
    detector_cols = int(np.ceil(len(detectors) / detector_rows))
    final_fig = make_subplots(
        rows=2 + detector_rows, cols=detector_cols, shared_xaxes=True, shared_yaxes=True,
        subplot_titles=["Overall"] + list(detectors),
        vertical_spacing=0.075, horizontal_spacing=0.025,
        specs=[
            [{"type": "bar", "rowspan": overall_rows, "colspan": detector_cols}, *[None for _c in range(detector_cols - 1)]],
            [None for _c in range(detector_cols)],
            *[[{"type": "bar"}for _c in range(detector_cols)] for _r in range(detector_rows)],
        ],
        print_grid=False,
    )
    
    # per-detector plots
    for i, det in enumerate(detectors):
        row, col = 1 + overall_rows + i // detector_cols, 1 + i % detector_cols
        bar_fig = single_bar_plot(data, metric, gt_annotator, det, include_annotators=include_annotators)
        bar_fig.for_each_trace(lambda trace: final_fig.add_trace(trace, row=row, col=col))
    final_fig.for_each_trace(lambda trace: trace.update(showlegend=False))
    # overall plot
    global_bar_fig = single_bar_plot(data, metric, gt_annotator, include_annotators=include_annotators)
    global_bar_fig.for_each_trace(lambda trace: final_fig.add_trace(trace, row=1, col=1))
    
    # update layout
    for r in range(detector_rows):
        final_fig.update_yaxes(title_text=metric.replace('_', ' ').title(), row=1 + overall_rows + r, col=1)
    final_fig.update_layout(
        width=900, height=750,
        title=f"Onset-Offset Comparisons (GT: {gt_annotator.upper()})",
        yaxis_title=metric.replace('_', ' ').title(),
        # move legend to bottom
        # legend=dict(orientation="h", yanchor="top", xanchor="center", xref='container', yref='container', x=0.5, y=0.85),
        showlegend=True,
    )
    return final_fig

## Results

In [6]:
METRIC = 'd_prime'
ANNOTATOR = 'RA'
INCLUDE_OTHER_ANNOTATORS = False

#### (0) Difference in Means
Check if there is a difference in mean performance across all detectors and trials.

In [31]:
_extract_data(metrics, METRIC, ANNOTATOR, include_annotators=INCLUDE_OTHER_ANNOTATORS).groupby(
    level=[peyes.constants.EVENT_STR, peyes.constants.CHANNEL_TYPE_STR]).mean().mean(axis=1).unstack(1)

channel_type,offset,onset
event,,
fixation,2.785907,1.597712
saccade,1.600489,3.034778


#### (1) Between-Detector Comparison

In [7]:
global_results = linear_mixed_effect(metrics, METRIC, ANNOTATOR, include_annotators=INCLUDE_OTHER_ANNOTATORS)
global_results.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                    Mixed Linear Model Regression Results
=============================================================================================================
Model:                              MixedLM                   Dependent Variable:                   d_prime  
No. Observations:                   560                       Method:                               REML     
No. Groups:                         7                         Scale:                                0.5993   
Min. group size:                    80                        Log-Likelihood:                       -688.2611
Max. group size:                    80                        Converged:                            Yes      
Mean group size:                    80.0                                                                     
-------------------------------------------------------------------------------------------------------------
                                                                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                                           2.786    0.462  6.031 0.000  1.881  3.691
event[T.saccade]                                                   -1.185    0.377 -3.148 0.002 -1.924 -0.447
channel_type[T.onset]                                              -1.188    0.431 -2.755 0.006 -2.034 -0.343
event[T.saccade]:channel_type[T.onset]                              2.622    0.650  4.036 0.000  1.349  3.896
pred Var                                                            1.463    1.121                           
pred x event[T.saccade] Cov                                        -0.899    0.812                           
event[T.saccade] Var                                                0.933    0.745                           
pred x channel_type[T.onset] Cov                                   -0.660    0.823                           
event[T.saccade] x channel_type[T.onset] Cov                        0.884    0.773                           
channel_type[T.onset] Var                                           1.243    0.977                           
pred x event[T.saccade]:channel_type[T.onset] Cov                   1.212    1.292                           
event[T.saccade] x event[T.saccade]:channel_type[T.onset] Cov      -1.539    1.243                           
channel_type[T.onset] x event[T.saccade]:channel_type[T.onset] Cov -1.732    1.409                           
event[T.saccade]:channel_type[T.onset] Var                          2.835    2.217                           
=============================================================================================================

"""

In [8]:
pd.concat([global_results.pvalues, global_results.pvalues <= 0.05], axis=1)

,0,1
Intercept,1.624756e-09,True
event[T.saccade],1.646499e-03,True
channel_type[T.onset],5.876782e-03,True
event[T.saccade]:channel_type[T.onset],5.433964e-05,True
pred Var,9.169509e-02,False
pred x event[T.saccade] Cov,1.525119e-01,False
event[T.saccade] Var,1.057907e-01,False
pred x channel_type[T.onset] Cov,2.998531e-01,False
event[T.saccade] x channel_type[T.onset] Cov,1.395194e-01,False
channel_type[T.onset] Var,1.003332e-01,False


#### (2) Across-Detector Post-Hoc (Dunn's Test)
##### (using the mean performance across all detectors)

In [9]:
across_detector_post_hoc = posthoc_dunn(metrics, METRIC, ANNOTATOR, "mean", include_annotators=INCLUDE_OTHER_ANNOTATORS)

across_detector_post_hoc

event                  fixation       saccade      
channel_type             offset onset  offset onset
event    channel_type                              
fixation offset              --   ***     ***  n.s.
         onset         0.000062    --    n.s.   ***
saccade  offset        0.000047   1.0      --   ***
         onset              1.0   0.0     0.0    --

In [49]:
z = across_detector_post_hoc.loc[("saccade", "onset")]
z.iloc[1], z.iloc[2]

(np.float64(2.590731014365712e-07), np.float64(1.8297240882439637e-07))

#### (3) Within-Detector Comparison

In [12]:
%%capture --no-display

exclude_detectors = [ANNOTATOR] if INCLUDE_OTHER_ANNOTATORS else [GT1, GT2]

per_detector_results = {
    det: linear_mixed_effect(metrics, METRIC, ANNOTATOR, det, include_annotators=INCLUDE_OTHER_ANNOTATORS)
    for det in metrics.columns.get_level_values(u.PRED_STR).unique() if not det in exclude_detectors
}

In [13]:
for det in sorted(per_detector_results.keys(), key=lambda dett: LABELER_PLOTTING_CONFIG[dett][0]):
    print(f"\n################################\nDetector: {det}")
    display(per_detector_results[det].summary())


################################
Detector: ivt


<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Mixed Linear Model Regression Results
============================================================================
Model:                   MixedLM        Dependent Variable:        d_prime  
No. Observations:        80             Method:                    REML     
No. Groups:              2              Scale:                     0.8510   
Min. group size:         40             Log-Likelihood:            -110.6105
Max. group size:         40             Converged:                 Yes      
Mean group size:         40.0                                               
----------------------------------------------------------------------------
                                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                          3.147    1.128  2.790 0.005  0.936  5.358
channel_type[T.onset]             -0.090    2.577 -0.035 0.972 -5.141  4.961
event Var                          2.503                                    
event x channel_type[T.onset] Cov -5.747                                    
channel_type[T.onset] Var         13.197                                    
============================================================================

"""


################################
Detector: ivvt


<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Mixed Linear Model Regression Results
===========================================================================
Model:                    MixedLM       Dependent Variable:       d_prime  
No. Observations:         80            Method:                   REML     
No. Groups:               2             Scale:                    0.7800   
Min. group size:          40            Log-Likelihood:           -106.5532
Max. group size:          40            Converged:                No       
Mean group size:          40.0                                             
---------------------------------------------------------------------------
                                  Coef.  Std.Err.   z   P>|z| [0.025 0.975]
---------------------------------------------------------------------------
Intercept                          3.108    0.597 5.204 0.000  1.937  4.279
channel_type[T.onset]              0.103    1.266 0.081 0.935 -2.379  2.584
event Var                          0.674    1.547                          
event x channel_type[T.onset] Cov -1.452    2.688                          
channel_type[T.onset] Var          3.128    4.917                          
===========================================================================

"""


################################
Detector: idt


...

<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Mixed Linear Model Regression Results
============================================================================
Model:                   MixedLM        Dependent Variable:        d_prime  
No. Observations:        80             Method:                    REML     
No. Groups:              2              Scale:                     0.6481   
Min. group size:         40             Log-Likelihood:            -100.2647
Max. group size:         40             Converged:                 No       
Mean group size:         40.0                                               
----------------------------------------------------------------------------
                                   Coef.  Std.Err.   z   P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                           3.054    1.872 1.632 0.103 -0.614  6.723
channel_type[T.onset]               0.631    2.859 0.221 0.825 -4.973  6.235
event Var                           6.975                                   
event x channel_type[T.onset] Cov -10.658                                   
channel_type[T.onset] Var          16.286                                   
============================================================================

"""


################################
Detector: nh


<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Mixed Linear Model Regression Results
============================================================================
Model:                   MixedLM        Dependent Variable:        d_prime  
No. Observations:        80             Method:                    REML     
No. Groups:              2              Scale:                     0.7858   
Min. group size:         40             Log-Likelihood:            -104.9777
Max. group size:         40             Converged:                 No       
Mean group size:         40.0                                               
----------------------------------------------------------------------------
                                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------------
Intercept                          2.026    0.141 14.332 0.000  1.749  2.303
channel_type[T.onset]              0.008    0.201  0.038 0.970 -0.385  0.401
event Var                          0.001                                    
event x channel_type[T.onset] Cov -0.001                                    
channel_type[T.onset] Var          0.002    0.130                           
============================================================================

"""


################################
Detector: remodnav


<class 'statsmodels.iolib.summary2.Summary'>
"""
                   Mixed Linear Model Regression Results
===========================================================================
Model:                   MixedLM        Dependent Variable:        d_prime 
No. Observations:        80             Method:                    REML    
No. Groups:              2              Scale:                     0.2742  
Min. group size:         40             Log-Likelihood:            -65.6712
Max. group size:         40             Converged:                 Yes     
Mean group size:         40.0                                              
---------------------------------------------------------------------------
                                  Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------
Intercept                         1.005    0.088 11.380 0.000  0.832  1.178
channel_type[T.onset]             0.953    0.458  2.081 0.037  0.055  1.851
event Var                         0.002                                    
event x channel_type[T.onset] Cov 0.027                                    
channel_type[T.onset] Var         0.392                                    
===========================================================================

"""

#### (4) Within Detector Post-Hoc (Dunn's Test)

In [18]:
posthoc_results = {
    det: posthoc_dunn(metrics, METRIC, ANNOTATOR, det, include_annotators=INCLUDE_OTHER_ANNOTATORS)
    for det in metrics.columns.get_level_values(u.PRED_STR).unique() if not det in exclude_detectors
}

for det in sorted(posthoc_results.keys(), key=lambda dett: LABELER_PLOTTING_CONFIG[dett][0]):
    print(f"\n################################\nDetector: {det}")
    is_significant = per_detector_results[det].pvalues['channel_type[T.onset]'] <= 0.05
    if is_significant:
        display(posthoc_results[det])
    else:
        print("Main Analysis Not Significant!")


################################
Detector: ivt
Main Analysis Not Significant!

################################
Detector: ivvt
Main Analysis Not Significant!

################################
Detector: idt
Main Analysis Not Significant!

################################
Detector: idvt
Main Analysis Not Significant!

################################
Detector: engbert
Main Analysis Not Significant!

################################
Detector: nh
Main Analysis Not Significant!

################################
Detector: remodnav


event                  fixation           saccade      
channel_type             offset     onset  offset onset
event    channel_type                                  
fixation offset              --      n.s.    n.s.   ***
         onset         0.290781        --    n.s.   ***
saccade  offset             1.0  0.132237      --   ***
         onset              0.0  0.000821     0.0    --

#### Summary Figure

In [19]:
W, H = 900, 750
yaxis_title = r'$d^{\prime}$'

In [23]:
FIG_NUM = 6
ANNOTATOR = 'RA'
IS_SUPPLEMENTARY = False

summary_fig1 = multiple_bar_plots(metrics, METRIC, ANNOTATOR, include_annotators=INCLUDE_OTHER_ANNOTATORS)
summary_fig1.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    
    # update yaxis titles
    yaxis_title=yaxis_title, yaxis2_title=yaxis_title, yaxis6_title=yaxis_title,
    
    legend=dict(orientation="h", yanchor="top", xanchor="center", xref='paper', yref='paper', x=0.12, y=0.99),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
)

save_fig(summary_fig1, FIG_NUM, ANNOTATOR, 'onset-offset-comparison', IS_SUPPLEMENTARY)
summary_fig1.show()

In [22]:
FIG_NUM = 10
ANNOTATOR = 'MN'
IS_SUPPLEMENTARY = True

summary_fig2 = multiple_bar_plots(metrics, METRIC, ANNOTATOR, include_annotators=INCLUDE_OTHER_ANNOTATORS)
summary_fig2.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    
    # update yaxis titles
    yaxis_title=yaxis_title, yaxis2_title=yaxis_title, yaxis6_title=yaxis_title,
    
    legend=dict(orientation="h", yanchor="top", xanchor="center", xref='paper', yref='paper', x=0.12, y=0.99),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
)

save_fig(summary_fig2, FIG_NUM, ANNOTATOR, 'onset-offset-comparison', IS_SUPPLEMENTARY)
summary_fig2.show()